# Intermediate Neural Network in Keras

Bulding an Intermediate Neural Network to classify MNIST digits.

#### Set seed for reproducibility

In [ ]:
import numpy as np
np.random.seed(42)

#### Load dependencies

In [ ]:
import os
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout, LeakyReLU
from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping

#### Load data

In [ ]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

#### Preprocess data
Flatten and normalise input data.

In [ ]:
X_train = X_train.reshape(60000, 784).astype('float32')
X_test = X_test.reshape(10000, 784).astype('float32')

In [ ]:
X_train /= 255
X_test /= 255

In [ ]:
# One-hot encoded categories
n_classes = 10
y_train = keras.utils.to_categorical(y_train, n_classes)
y_test = keras.utils.to_categorical(y_test, n_classes)

#### ReLUs

In [ ]:
from keras import backend as K
from keras.engine import Layer
from keras.utils.generic_utils import get_custom_objects

class SineReLU(Layer):

    def __init__(self, epsilon=0.0055, **kwargs):
        super(SineReLU, self).__init__(**kwargs)
        self.supports_masking = True
        self.epsilon = K.cast_to_floatx(epsilon)

    def build(self, input_shape):
        self.scale = np.exp(np.sqrt(np.pi))
        super(SineReLU, self).build(input_shape)

    def call(self, Z):
        m = self.epsilon * (K.sigmoid(K.sin(Z)) - K.sigmoid(K.cos(Z)) * self.scale)
        A = K.maximum(m, Z)
        return A

    def get_config(self):
        config = {'epsilon': float(self.epsilon)}
        base_config = super(SineReLU, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

    def compute_output_shape(self, input_shape):
        return input_shape

#### Design Neural Network architecture

In [ ]:
model = Sequential()
model.add(Dense(128, input_shape = (784,)))
model.add(SineReLU(epsilon=0.0083))
model.add(Dropout(0.2))

model.add(Dense(256))
model.add(SineReLU(epsilon=0.0083))
model.add(Dropout(0.3))

model.add(Dense(1024))
model.add(SineReLU(epsilon=0.0083))
model.add(Dropout(0.5))
model.add(Dense(10, activation = 'softmax'))

In [ ]:
model.summary()

#### Callbacks

In [ ]:
modelCheckpoint = ModelCheckpoint(monitor='val_acc', filepath='model_output/weights-deepnet-mnist.hdf5',
                                               save_best_only=True, mode='max')
earlyStopping = EarlyStopping(monitor='val_acc', mode='max', patience=20)


if not os.path.exists('model_output'):
    os.makedirs('model_output')
    
tensorboard = TensorBoard("../logs/deepnet-mnist-relus-native-VII")

#### Configure model

In [ ]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

#### Train!

In [ ]:
history = model.fit(X_train, y_train, batch_size = 128, epochs = 50, verbose = 1,
          validation_split = 0.1, callbacks=[modelCheckpoint, earlyStopping])#, tensorboard])

#### Test Predictions

In [ ]:
saved_model = keras.models.load_model('model_output/weights-deepnet-mnist.hdf5', custom_objects={'SineReLU': SineReLU})
predictions = saved_model.predict_classes(X_test, verbose = 2)
np.std(history.history['loss'])

#### Test Final Accuracy

In [ ]:
final_loss, final_acc = model.evaluate(X_test, y_test, verbose = 1)
print("Final loss: {0:.4f}, final accuracy: {1:.4f}".format(final_loss, final_acc))

In [ ]:
#Final loss: 0.0765, final accuracy: 0.9833 -> SineReLU (0.0083); STD loss => 0.05375531819714868
#Final loss: 0.0745, final accuracy: 0.9827 -> SineReLU (0.0080); STD loss => 0.05386899405890265
#Final loss: 0.0859, final accuracy: 0.9819 -> SineReLU (0.0085); STD loss => 0.053545432633176175
#Final loss: 0.0729, final accuracy: 0.9819 -> ReLU; STD loss => 0.057364919558005455
#Final loss: 0.0761, final accuracy: 0.9823 -> ReLU; STD loss => 0.057064730691239844
#Final loss: 0.0823, final accuracy: 0.9829 -> ReLU; STD loss => 0.05736969016884351